<a href="https://colab.research.google.com/github/polakamal/Arabic-Semantic-Error-Detection-and-Correction/blob/main/LSTMproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jiwer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Sun Jun  6 22:19:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from nltk import  word_tokenize,sent_tokenize
import re
import nltk
import pandas as pd
import numpy as np
import string
pd.set_option("display.max_colwidth", -1)
nltk.download('punkt')
punctuations = set(string.punctuation)
punctuations.update('‘','’','،','؛','؟')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [ ]:
def extract_data(file_name):
  sentence = []
  corrected= []
  f = open(file_name, "r", encoding="utf8")
  for l in f:
        if l.startswith('S '):
            sentence.append(' '.join(l.split()[1:]))
            corrected.append("[br]")
        if l.startswith('A '):
            corrected.append(' '.join(l.split()[1:]))
  data = extract_corrected_sent(sentence,corrected)
  return  data  

In [ ]:
def extract_corrected_sent(sentence,corrected_sent):
  data=[]
  count=0
  counter=0
  para =''
  sample= ''
  add_b_action = 'Add_before'
  merge_action='Merge'
  split_action='Split'
  delete_action='Delete'
  other_action='Other'
  add_a_action ='Add_after'
  delete_token = 'delete_token'
  add_token_before = 'add_token_before'
  add_token_after='add_token_after'
  splitInsert4tokens='splitInsert4tokens'
  splitInsert3tokens='splitInsert3tokens'
  split_merge='split_merge'
  merge_split='merge-split'
  for i in corrected_sent:
     if i == '[br]':
         counter=0
         para= sentence[count]
         if count==0:
            data.append([sentence[count],sample])
         else:
            data.append([sentence[count-1],sample]) 
         count=count+1
         continue     
     
     fields=i.split('|||')
     start_offset = int(fields[0].split()[0])
     end_offset = int(fields[0].split()[1])
     word= fields[2]
     action= fields[1]
     #print("before :    "+para )
     #print(counter)
     #print(action)   
     sample = correted_sentence_data(para ,action ,start_offset+counter,end_offset+counter,word)
     if add_a_action ==action or add_b_action ==action or split_action ==action or add_token_before==action or add_token_after==action or split_action.lower()==action:
       counter =counter+1
     elif  merge_action==action or delete_action==action or delete_token==action or merge_action.lower()==action :
         counter =counter-1 
     elif other_action==action or splitInsert4tokens ==action or splitInsert3tokens ==action or split_merge==action:
         if  end_offset - start_offset == 1 :
            counter =counter +(len(word_tokenize(word))-1)
         elif  end_offset - start_offset != len(word_tokenize(word)):
           counter =counter - (end_offset - start_offset)
           counter =counter + len(word_tokenize(word))

     para = sample  
     #print("after :    "+para )
    
     #print("________________________________________________________________________________________________ " )

  return data

In [ ]:
def correted_sentence_data(para ,action ,start_offset,end_offset,changed_word):
  action1 ='Add_before'
  action2='Merge'
  action3='Split'
  action4='Delete'
  action5 = 'Edit'
  action6='Other'
  action7='Move'
  action8 ='Add_after'
  delete_token = 'delete_token'
  add_token_before = 'add_token_before'
  add_token_after='add_token_after'
  move_before='move_before'
  splitInsert4tokens='splitInsert4tokens'
  splitInsert3tokens='splitInsert3tokens'
  split_merge='split_merge'
  merge_split='merge-split'
  sent= ''
  counter = 0
  sample = word_tokenize(para)
  if action1 == action or add_token_before==action:
           sample.insert(start_offset ,changed_word)
           sent= ' '.join(word for word in sample)
           
  elif action6 ==action  or splitInsert4tokens ==action or splitInsert3tokens ==action or split_merge==action or merge_split==action:
    if  end_offset - start_offset > 1 :
      for i in range(end_offset - start_offset):
           sample.pop(start_offset)
        
      sample.insert(start_offset,changed_word)
      sent= ' '.join(word for word in sample)
    else:  
           sample.pop(start_offset)
           sample.insert(start_offset,changed_word)
           sent= ' '.join(word for word in sample)
 
  elif action2 == action  or action7==action or move_before==action or action2.lower() == action:
           sample.pop(start_offset)
           sample.pop(start_offset)
           sample.insert(start_offset,changed_word)
           sent= ' '.join(word for word in sample)
  elif  action3 == action or action3.lower()==action  :
           sample.pop(start_offset)
           sample.insert(start_offset,changed_word)
           sent= ' '.join(word for word in sample)
  elif action4 == action or delete_token==action:
           sample.pop(start_offset)
           sent= ' '.join(word for word in sample) 
  elif action5 == action or action5.lower()==action: 
           
           sample[start_offset] =changed_word
           sent= ' '.join(word for word in sample)
           #print("inside the edit",sent)
  elif action8==action or add_token_after==action:
          sample.insert(end_offset ,changed_word)
          sent= ' '.join(word for word in sample)    
           #print("________________________________________________________________________________________________ " )
  #print("after :    "+sent )     
  return sent 

In [ ]:
train = extract_data("/content/drive/MyDrive/NlpProject/data/QALB-Train2014.m2")

In [ ]:
train.pop(0)

['الى التعليق رقم 2 اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور . فليتهم يفعلونها ولو مرة . ولنا نحن كشعوب مسؤولية كذالك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر .',
 '']

In [ ]:
df_train =  pd.DataFrame(train ,columns=['sentence','corrected'])


In [ ]:
vald = extract_data("/content/drive/MyDrive/NlpProject/data/QALB-Dev2014.m2")
test = extract_data("/content/drive/MyDrive/NlpProject/data/QALB-Test2014.m2")

In [ ]:
vald.pop(0)
test.pop(0)

['لا زال كبير الشبيحه يظن ان ارواح وآلام الناس اقل كلفه من تخليه عن منصبه ، فلذلك اذا كان السوريون لا يرتضون بهذه المعادله المهينه ، فعليهم ان يهبوا هبه قويه واحده وياخذو حقوقهم من هذه العصابه عنوه ، اننا يا أحبائي ندفع ثمن اكثر من اربعين عام ومن الخنوع والذل والثمن سيكون غاليا ولكنه يستأهل هذه التضحيات',
 '']

In [ ]:
df_vald=  pd.DataFrame(vald ,columns=['sentence','corrected'])
df_test =  pd.DataFrame(test ,columns=['sentence','corrected'])

In [ ]:
df_vald.size,df_test.size

(2032, 1934)

In [ ]:
def preprocessing(text):
  for punctuation in punctuations:
        text = text.replace(punctuation,'') 
  text=re.sub('\d+','',text)
  text=re.sub(r'\. \.+','',text)
  text=re.sub(r'[\`|\"|\(|\)|\%|\[|\]\_\-\*\&\@\?\!]','',text)
  text= re.sub(' +',' ',text)
  text=text.strip()
  return text 

In [ ]:
df_train['sentence'] = df_train['sentence'].apply(lambda x: preprocessing( x ) )
df_train['corrected'] = df_train['corrected'].apply(lambda x: preprocessing( x ) )

In [ ]:
df_vald['sentence'] = df_vald['sentence'].apply(lambda x: preprocessing( x ) )
df_vald['corrected'] = df_vald['corrected'].apply(lambda x: preprocessing( x ) )

In [ ]:
df_test['sentence'] = df_test['sentence'].apply(lambda x: preprocessing( x ) )
df_test['corrected'] = df_test['corrected'].apply(lambda x: preprocessing( x ) )

In [ ]:
df_train.head(10)

,sentence,corrected
0,الى التعليق رقم اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذالك تتمثل في مساندة اخواننا في الصين بمقاطعة...,إلى التعليق رقم أكيد أن للحكام العرب والمسلمين مسؤولية يتمثل أدناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذلك تتمثل في مساندة إخواننا في الصين بمقاطعة...
1,نحن ببالغ الاسى نعزي ضحايا الحادث الاليم الذي ذهب ضحيته حوالي ما بين جريح ومتوفي واطالب من سلطات بلادي ان تطبق القانون بصرامة وتضرب بيد من حديد مثل هؤلاء المجرمين والمتهورين والمتسببين في مثل هذه ...,نحن ببالغ الأسى نعزي ضحايا الحادث الأليم الذي ذهب ضحيته حوالي ما بين جريح ومتوفي وأطالب من سلطات بلادي أن تطبق القانون بصرامة وتضرب بيد من حديد مثل هؤلاء المجرمين والمتهورين والمتسببين في مثل هذه ...
2,للاسف ان الدم المهدور عراقي لكن بشكل عام لماذا الاتهام من قبل الاعلام الغربي والعربي للقاعدة والتطرف والناس المتسللة من دول الجوار فقط هل اصبح العراقيون جميعهم بلا دم واحساس وادناب للامريكان ولا ا...,للأسف إن الدم المهدور عراقي لكن بشكل عام لماذا الاتهام من قبل الإعلام الغربي والعربي للقاعدة والتطرف والناس المتسللة من دول الجوار فقط هل أصبح العراقيون جميعهم بلا دم وإحساس وأذنابا للأمريكان ولا ...
3,واصل الثوار أنتصاراتهم وحررو كل حقول النفط السورية لهذا السبب قام النظام اللبناني الخائن بمساعدة نظام الاسد بالبنزين والمازوت منتهكآ العقوبات المفروضة على النظام ولكن الثوار قررو قطع الطريق بين دم...,واصل الثوار انتصاراتهم وحرروا كل حقول النفط السورية لهذا السبب قام النظام اللبناني الخائن بمساعدة نظام الأسد بالبنزين والمازوت منتهكا العقوبات المفروضة على النظام ولكن الثوار قرروا قطع الطريق بين ...
4,يا حرام على ما يدعو نفسهم شبيحة الأسد وحزب الله بعد ان ارسلهم عميل الصهاينة المستعرب خامنئي الجزار لقتل أهلهم وخدمة لمصالح وطموح ايران بالسيطرة على بلاد الشام ولكنهم أكتشفوا ان موتهم ونهايتهم في د...,يا حرام على من يدعون نفسهم شبيحة الأسد وحزب الله بعد أن أرسلهم عميل الصهاينة المستعرب خامنئي الجزار لقتل أهلهم وخدمة لمصالح وطموح إيران بالسيطرة على بلاد الشام ولكنهم اكتشفوا أن موتهم ونهايتهم في ...
5,اليوم المظاهرات الحاشدة تعتبر جمعة مفصلية تاريخية حيث حظي المجلس بتاييد شعبي واسع وايضا ظهر ان بشار والة القتل عجزت عن مواجهة الشعب السوري بل ان باعتقادي ان بشار سينهار قريبا فالجزيرة نقلت عن ميدف...,اليوم المظاهرات الحاشدة تعتبر جمعة مفصلية تاريخية حيث حظي المجلس بتأييد شعبي واسع وأيضا ظهر أن بشار وآلة القتل عجزت عن مواجهة الشعب السوري بل إن باعتقادي أن بشار سينهار قريبا فالجزيرة نقلت عن ميدف...
6,وقف اطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف اطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف اطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف اطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة,وقف إطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف إطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف إطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة وقف إطلاق الصواريخ من غزة مقابل رفع الحصار عن غزة
7,إلى الإخوة في الجزيرة هؤلاء شهداء وهم خيرة الشهداء وهم ليسوا قتلى عاديين أرجو الله أن يكونوا في منزلة خيرة شهداء هذه الأمة أمثال حمزة وعمر وعلي والحسين نرجو وصفهم بالشهداء وليسوا بالقتلى وذلك إحتر...,إلى الإخوة في الجزيرة هؤلاء شهداء وهم خيرة الشهداء وهم ليسوا قتلى عاديين أرجو الله أن يكونوا في منزلة خيرة شهداء هذه الأمة أمثال حمزة وعمر وعلى والحسين نرجو وصفهم بالشهداء وليس بالقتلى وذلك احترام...
8,مصير جميع الإرهابيين السحق على يد الجيش التركي ومن يقول لماذا لايقوم الجيش التركي بالقضاء التام على الإرهابيين من حزب العمال كما فعل صدام أقول له أن الجيش التركي يراعي ظروف المنطقة واختباء الإرهاب...,مصير جميع الإرهابيين السحق على يد الجيش التركي ومن يقول لماذا لا يقوم الجيش التركي بالقضاء التام على الإرهابيين من حزب العمال كما فعل صدام أقول له أن الجيش التركي يراعي ظروف المنطقة واختباء الإرها...
9,ثورة الآخرين على ارض الشام حيث ان الجيش الحر يستجلب ما هب ودب من مقاتلين اجانب ليقاتلوا نيابة عن الشعب السوري وهذه الثورة تعتبر فاشلة لانها لا توصل الا لهدف تدمير سوريا من الداخل لان الشعب السوري ...,ثورة الآخرين على أرض الشام حيث أن الجيش الحر يستجلب ما هب ودب من مقاتلين أجانب ليقاتلوا نيابة عن الشعب السوري وهذه الثورة تعتبر فاشلة لأنها لا توصل إلا لهدف تدم

In [ ]:
sents=[]
for i in df_train['sentence']:
  sent = word_tokenize(i)
  sents.append(sent)

In [ ]:
correct=[]
for i in df_train['corrected']:
  sent = word_tokenize(i)
  correct.append(sent)

In [ ]:
max_text_len = max( [len(i) for i in sents] )

In [ ]:
max_text_len

169

In [ ]:
max_summary_len = max( [len(i) for i in correct] )

In [ ]:
max_summary_len

95

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM,Input,TimeDistributed,Bidirectional,Concatenate
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from keras import backend as K 
from keras import optimizers
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
df_train['corrected'] = df_train['corrected'].apply(lambda x : 'sostok '+ x + ' eostok')
df_vald['corrected']= df_vald['corrected'].apply(lambda x : 'sostok '+ x + ' eostok')
df_test['corrected']= df_test['corrected'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(df_vald['sentence'].append(df_train['sentence'])))

In [ ]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 67.40772673153508
Total Coverage of rare words: 7.761759539810238


In [ ]:
x_tokenizer = Tokenizer(num_words=tot_cnt) 
x_tokenizer.fit_on_texts(list(df_vald['sentence'].append(df_train['sentence'])))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(df_train['sentence']) 
x_val_seq   =   x_tokenizer.texts_to_sequences(df_vald['sentence'])

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

Size of vocabulary in X = 146143


In [ ]:
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(df_vald['corrected'].append(df_train['corrected'])))

In [ ]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 77.55060146980334
Total Coverage of rare words: 10.870317326704372


In [ ]:
y_tokenizer = Tokenizer(num_words=tot_cnt) 
y_tokenizer.fit_on_texts(list(df_vald['corrected'].append(df_train['corrected'])))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(df_train['corrected']) 
y_val_seq   =   y_tokenizer.texts_to_sequences(df_vald['corrected']) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

Size of vocabulary in Y = 146143


In [ ]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)
        
y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [ ]:
K.clear_session()

latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 105,091,043
Trainable params: 105,091,043
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=30,batch_size=32,validation_data= ([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]) )

Epoch 1/30
607/607 [==============================] - 334s 541ms/step - loss: 5.1128 - accuracy: 0.4856 - val_loss: 4.6260 - val_accuracy: 0.4957
Epoch 2/30
607/607 [==============================] - 328s 541ms/step - loss: 4.4859 - accuracy: 0.5002 - val_loss: 4.4678 - val_accuracy: 0.5039
Epoch 3/30
607/607 [==============================] - 328s 540ms/step - loss: 4.2702 - accuracy: 0.5112 - val_loss: 4.3393 - val_accuracy: 0.5142
Epoch 4/30
607/607 [==============================] - 328s 541ms/step - loss: 4.0574 - accuracy: 0.5214 - val_loss: 4.2492 - val_accuracy: 0.5200
Epoch 5/30
607/607 [==============================] - 327s 539ms/step - loss: 3.8581 - accuracy: 0.5290 - val_loss: 4.2071 - val_accuracy: 0.5230
Epoch 6/30
607/607 [==============================] - 327s 539ms/step - loss: 3.6677 - accuracy: 0.5365 - val_loss: 4.1711 - val_accuracy: 0.5259
Epoch 7/30
607/607 [==============================] - 327s 539ms/step - loss: 3.4831 - accuracy: 0.5433 - val_loss: 4.1627 -

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
target_word_index = y_tokenizer.word_index
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=y_tokenizer.index_word

In [ ]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_text_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString


In [ ]:
print(decode_sequence(x_tr[0].reshape(1,max_text_len)))

 إلى التعليق رقم إن لم يكن مستغربا أن تنبري زمرة شهداء الذين يكتبون وينتقدون الجزيرة ويدافعون عن الظلم المسلط عليهم باسم الإسلام والمسلمين إلى متى نقول ونصنع لأنفسنا جوا خياليا نعيش في الأرض فسادا أين كنتم لا نريد من أي دولة مسلمة التي نشاهدها الله بإذن الله


In [ ]:
print("Original text:",seq2text(x_tr[0]))
print("corrected:",seq2summary(y_tr[0]))

Original text: الى التعليق رقم اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذالك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر 
corrected: إلى التعليق رقم أكيد أن للحكام العرب والمسلمين مسؤولية يتمثل أدناها في استدعاء السفراء في الصين للتشاور فليتهم يفعلونها ولو مرة ولنا نحن كشعوب مسؤولية كذلك تتمثل في مساندة إخواننا في الصين بمقاطعة البضائع الصينية وليتنا نفعلها ولو ثلاثة أشهر 


In [ ]:
print(decode_sequence(x_val[0].reshape(1,max_text_len)))

 سبحان الله يرصدون أموال طائلة لمن يدمر على يد الشبيحة الذين يكتبون بأسماء من الكلية الحربية على الجيش السوري الحر بالموزاة مع بقية المناطق العربية لاتخاذ الوقت لزيارة ماهر الأسد بزيارة جبل الزاوية أكثر من نصف قرن من الجيش السوري بساحة مع من يقاتل بجيش المهدي إلى الآن


In [ ]:
print("Original text:",seq2text(x_val[0]))
print("corrected:",seq2summary(y_val[0]))

Original text: سبحان الله الحكام العرب سيموت على الكرسي ليضهر أنه عنيد وقوي لوكان بشار يحب أرضه أو شعبه لخرج من الحكم شفقة ورحمة ببلد ضاع هنا زال قناع هذا الرئيس اللذي خيب ظن شعبه والشعوب المسلمة كل مال السورين نفق في شراء سلاح ليقتل به شتانا وحكام أوربا الذين يتركون الكرسي لمجرد فتنة بسيطة لحبهم لبلدهم 
corrected: سبحان الله الحكام العرب سيموت على الكرسي ليظهر أنه عنيد وقوي لو كان بشار يحب أرضه أو شعبه لخرج من الحكم شفقة ورحمة ببلد ضاع هنا زال قناع هذا الرئيس الذى خيب ظن شعبه والشعوب المسلمة كل مال السورين نفق في شراء سلاح ليقتل به شتان وحكام أوربا الذين يتركون الكرسي لمجرد فتنة بسيطة لحبهم لبلدهم 


In [ ]:
df_vald['corrected'][0]

'sostok سبحان الله الحكام العرب سيموت على الكرسي ليظهر أنه عنيد وقوي لو كان بشار يحب أرضه أو شعبه لخرج من الحكم شفقة ورحمة ببلد ضاع هنا زال قناع هذا الرئيس الذى خيب ظن شعبه والشعوب المسلمة كل مال السورين نفق في شراء سلاح ليقتل به شتان وحكام أوربا الذين يتركون الكرسي لمجرد فتنة بسيطة لحبهم لبلدهم eostok'

In [ ]:
inputs=[]
for i in  range (len(x_val)):
    input =decode_sequence(x_val[i].reshape(1,max_text_len))
    inputs.append(input)

In [ ]:
from jiwer import wer

In [ ]:
scores_wer=[]
for i in range (len(inputs)):
  w_e_r = wer(inputs[i],df_vald['corrected'][i]) 
  scores_wer.append(w_e_r)

In [ ]:
np.mean(scores_wer)

1.065687944852582